In [1]:
import neuropythy as ny
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib as plt
import ipyvolume as ipv

Add your key and secret code in the next cell

In [2]:
ny.config['hcp_credentials'] = ('key', 'secret')
ny.config['hcp_auto_download'] = True

Set the number of ICAs you are using in the next cell (replace X with 15, 25, 50, 100 or 200)

In [3]:
icanum = 15

In [4]:
sub = ny.hcp_subject(100206)
lh_hemi_native = sub.lh
rh_hemi_native = sub.rh
lh_hemi_LR32k = sub.hemis['lh_LR32k']
rh_hemi_LR32k = sub.hemis['rh_LR32k']

In [5]:
cii_filename = '~/data/connectivityml/HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d'+str(icanum)+'.ica/melodic_IC.dscalar.nii'
cii_obj = ny.load(cii_filename)

In [6]:
(lh_data, rh_data, subvox_data) = ny.hcp.cifti_split(cii_obj)

Choose a percentile--how many vertices from the original ICA map do you want to keep? (90th, 95th or 99th work well)

In [7]:
cpercentile = 95

In [8]:
lh_thresh_list = list()
for i in range(0,icanum):
    myrow = lh_data[i]
    lh_thresh_list.append(np.nanpercentile(myrow, cpercentile))

In [9]:
rh_thresh_list = list()
for i in range(0,icanum):
    myrow = rh_data[i]
    #print(i, mymax, mymin, np.nanpercentile(myrow, 95))
    rh_thresh_list.append(np.nanpercentile(myrow, cpercentile))
    #print(np.nanpercentile(myrow, 90))
    #plt.hist(lh_data[i])

In [10]:
lh_labels = np.zeros(32492, dtype='int')
for (ii,myrow) in enumerate(lh_data):
    mycutoff = lh_thresh_list[ii]
    mylabel = ii + 1 # we want labels to start at 1, not 0
    lh_labels[myrow > mycutoff] = mylabel

In [11]:
lh_labels

array([ 0,  0, 13, ...,  5,  5,  5])

In [18]:
rh_labels = np.zeros(32492, dtype='int')
# for (ii,myrow) in enumerate(lh_data):
for (ii,myrow) in enumerate(rh_data):
    mycutoff = rh_thresh_list[ii]
    mylabel = ii + 1 # we want labels to start at 1, not 0
    rh_labels[myrow > mycutoff] = mylabel

In [19]:
rh_labels

array([ 2,  0, 13, ...,  5,  7,  5])

In [20]:
lh_hemi_LR32k = sub.hemis['lh_LR32k']
rh_hemi_LR32k = sub.hemis['rh_LR32k']
# We can make an ipyvolume figure to plot both hemispheres on:
fig = ipv.figure()
# Then plot each hemisphere using whichever ICA component we
# want to visualize:
ny.cortex_plot(lh_hemi_LR32k, surface='very_inflated',
               color=lh_labels, cmap='hot', figure=fig)
ny.cortex_plot(rh_hemi_LR32k, surface='very_inflated',
               color=rh_labels, cmap='hot', figure=fig)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

In [ ]:
# Explore Gephi
edge_df = pd.DataFrame(columns = ['Node1', 'Node2'])
edge_df['Node1'] = node1list
edge_df['Node2'] = node2list